In [1]:
import numpy as np
np.set_printoptions(suppress=True, precision=5)
import time
import pandas as pd

In [2]:
query_scores = np.fromfile("native-queries/native-query-scores.bin", dtype='f')
query_scores.shape

(6540000,)

In [3]:
print(query_scores[0:100])

[1.      0.86154 0.76303 0.70082 0.69154 0.63187 0.63078 0.597   0.39991
 0.39806 0.34944 0.3238  0.31499 0.2983  0.24113 0.23587 0.233   0.22473
 0.22309 0.21943 0.20288 0.19483 0.19241 0.18163 0.17281 0.17081 0.16959
 0.16804 0.16621 0.16207 0.16194 0.15713 0.15247 0.14817 0.1462  0.14411
 0.14166 0.13771 0.13493 0.13477 0.13264 0.13195 0.12906 0.12826 0.12664
 0.12257 0.12221 0.12177 0.11971 0.11966 0.11786 0.11386 0.11345 0.11311
 0.11298 0.1124  0.11113 0.10563 0.10419 0.10283 0.10262 0.10107 0.1008
 0.10015 0.09225 0.09151 0.08893 0.08851 0.08758 0.08668 0.0863  0.08616
 0.08545 0.08544 0.08505 0.08456 0.08343 0.08306 0.08281 0.08263 0.08257
 0.08188 0.08015 0.07971 0.07845 0.07843 0.07547 0.07515 0.07509 0.07455
 0.07315 0.07139 0.07137 0.07098 0.07012 0.06991 0.06914 0.06889 0.06855
 0.06825]


In [4]:
# reshape np array into 327 x 20000 array
query_scores = query_scores.reshape(327, 20000)

In [5]:
query_frame_ids = np.fromfile("native-queries/native-query-frame-IDs.bin", dtype='i')
query_frame_ids.shape

(6540000,)

In [6]:
print(query_frame_ids[0:100])

[ 5697  5645 18842 10847  4872 15282  5644 11257  3449  5380 15283     1
  4489 17026 10845   965  5643 18845 12903  2241 19246  2226 15342  3245
   964  8758 12083 10711 19007 14892 15213 16454  4493 16475  3725  7890
  4927   498 10710   452  8128 14617  2564   963 19121  2810 14099  3556
 13992  6988   980 18702  2454 14890 13953  2663 16117  8347  2453 14374
  9795  4375   729  5819  9391 12902 10846 15811  7889 16186 10848  3451
 17975 19951 18358 16476 18843 18686  6129  7380 12721  5267  9626 17687
 12334 15387 14540 14911  3622  4589 12085  7879 10496  3482 14615  6905
 10152 19706  3481  9759]


In [7]:
# reshape np array into 327 x 20000 array
query_frame_ids = query_frame_ids.reshape(327, 20000)

In [8]:
frame_features = np.fromfile("native-queries/frame-features.bin", dtype='f')
frame_features.shape

(2560000,)

In [9]:
print(frame_features[0:100])

[ 0.16691  0.56416  0.32702 -0.11217  0.19371  0.5717   0.1079  -0.04373
  0.22609  0.0775   0.14043  0.08696  0.04192 -0.00181  0.02947 -0.05099
 -0.02955 -0.00342  0.06661  0.03804  0.01074  0.07426  0.05218  0.01714
 -0.05492  0.04126 -0.06568  0.05996 -0.03729 -0.06075  0.03306 -0.02322
 -0.01536 -0.00883  0.00342  0.01878  0.04251  0.02456 -0.0076   0.01512
  0.02498 -0.02595 -0.01833 -0.00277 -0.00717  0.00178 -0.00322  0.00384
  0.00497  0.01047 -0.0292   0.01857  0.01496  0.01483  0.03119  0.02505
 -0.03294  0.02223 -0.0336   0.01726 -0.01783  0.00684  0.01794 -0.00201
 -0.0444  -0.01943 -0.01307 -0.00856 -0.00216  0.00839  0.00798  0.0165
  0.02071 -0.00376  0.01017  0.01088  0.00909 -0.0092  -0.00007 -0.03174
 -0.05065  0.03829  0.00424  0.00299  0.01122 -0.01765  0.00512 -0.00211
 -0.00814 -0.04114 -0.01271  0.01588 -0.01666  0.00565  0.01208 -0.02558
 -0.01413 -0.01899 -0.02829 -0.01703]


In [10]:
# reshape np array into 20000 x 128 array
frame_features = frame_features.reshape(20000, 128)

In [75]:
# SOM Implementation

class SOM:
    
    def __init__(self, som_size, data, sigma, lambda_, learning_rate, iterations):
        
        # sigma - radius of a neighborhood
        self.som_size = som_size
        self.data = data
        self.sigma = sigma
        self.learning_rate = learning_rate
        # number of iterations
        self.iterations = iterations
        # lambda - time constant. used to update (= decay) the radius and learning rate
        self.lambda_ = lambda_
        # intialize SOM with random weights
        self.som = np.random.random((som_size, som_size, data.shape[1]))
       
    def update_learning_rate(self, i):
        return self.learning_rate * np.exp(-i / self.lambda_)

    def update_sigma(self, i):
        return self.sigma * np.exp(-i / self.lambda_)

    def neighborhood_function(self, sigma, dist):
        return np.exp(-(dist ** 2) / (2 * (sigma ** 2)))
    
    # calculating euclidean distance
    def euclidean_dist(self, x, y):
        return np.sqrt(np.sum((x - y) **2))
    
    # finding Best Matching Unit (TODO later: consdier ranking)
    def find_bmu(self, vector):
        
        min_dist = 1e+10
        for i in range(self.som.shape[0]):
            for j in range(self.som.shape[1]):
                dist = self.euclidean_dist(self.som[i, j], vector)
                if dist < min_dist:
                    min_dist = dist
                    bmu_index = np.array([i, j])

        return bmu_index
                    
    def train(self):
        
        start_time = time.time()
        
        for t in range(self.iterations):

            # randomly select an input row, find BMU
            rand = np.random.randint(0, self.data.shape[0])
            random_input = self.data[rand, :]
            bmu_index = self.find_bmu(random_input)
            
            # update learning rate and sigma with current time
            learning_rate = self.update_learning_rate(t)
            sigma = self.update_sigma(t)
            
            # update the weights of BMU and its neighborhood
            for i in range(self.som.shape[0]):
                for j in range(self.som.shape[1]):
                    som_node = np.array([i,j])
                    dist = self.euclidean_dist(som_node, bmu_index)
                    self.som[i, j] = self.som[i,j] + learning_rate * self.neighborhood_function(sigma, dist) * (random_input - self.som[i,j])

        print("Training complete. Time elapsed: %s seconds" % (time.time() - start_time))

    def node2image(self, filepaths):
        
        # run find_bmu to obtain BMU for each feature vector
        
        bmu_list = []
        for vec in self.data:
            bmu_list.append(self.find_bmu(vec))
        
        # storing the result in a dataframe. 
        # i-th row of the dataframe (ex: 4 1) is associated with the BMU coordinates of the i-th feature vector  
        
        data_ = pd.DataFrame(bmu_list, columns=['BMU_x','BMU_y'])
        
        # merge the 2 dataframes
        
        df = pd.concat([filepaths, data_], axis=1, join='inner')

        # now we have:
        
        # filename ID BMU_x BMU_y
        # ........ rows ........ 
        
        return df
        
    #TODO:
    def select_images(self, df):
        
        for i in range(self.som_size):
            for j in range(self.som_size):
                tmp = df.loc[(df.BMU_x == i) & (df.BMU_y == j)]
                
                print("df for i = {} & j = {}: {}".format(i, j, tmp.head()))

In [76]:
# training SOM with size 5 x 5
size = 5
som = SOM(som_size=5, data=frame_features, sigma=1, lambda_=1e2, learning_rate=0.5, iterations=100)
som.train()

Training complete. Time elapsed: 0.07340717315673828 seconds


In [77]:
# importing frame-ID-to-filepath.csv
filepaths = pd.read_csv(r"C:\Users\KWP\bachelor_thesis\native-queries\frame-ID-to-filepath.csv", sep=' ', names=['filename', 'ID'])                                                                                         

In [78]:
df = som.node2image(filepaths)

                                                filename     ID  BMU_x  BMU_y
0      v00000_s00000(f000000-f000013)_g00000_f000000.jpg      0      1      2
1      v00000_s00017(f002103-f002959)_g00050_f002800.jpg      1      0      0
2      v00000_s00029(f006081-f006173)_g00100_f006150.jpg      2      2      1
3      v00001_s00022(f002857-f002924)_g00035_f002907.jpg      3      1      0
4      v00001_s00053(f007125-f007189)_g00085_f007162.jpg      4      2      1
5      v00002_s00034(f002452-f002503)_g00046_f002487.jpg      5      1      2
6      v00002_s00062(f004514-f004638)_g00096_f004585.jpg      6      2      1
7      v00002_s00095(f006787-f006883)_g00146_f006833.jpg      7      1      1
8      v00002_s00129(f009078-f009146)_g00196_f009140.jpg      8      0      2
9      v00002_s00160(f010979-f011043)_g00246_f011028.jpg      9      0      1
10     v00003_s00028(f003123-f003198)_g00043_f003176.jpg     10      1      1
11     v00004_s00014(f002349-f002488)_g00027_f002397.jpg     11 

In [79]:
som.select_images(df)

df for i = 0 & j = 0:                                              filename  ID  BMU_x  BMU_y
1   v00000_s00017(f002103-f002959)_g00050_f002800.jpg   1      0      0
21  v00008_s00003(f000145-f005206)_g00015_f001728.jpg  21      0      0
22  v00009_s00001(f000050-f000405)_g00008_f000350.jpg  22      0      0
23  v00009_s00010(f002308-f002615)_g00058_f002550.jpg  23      0      0
24  v00009_s00016(f004066-f004635)_g00108_f004425.jpg  24      0      0
df for i = 0 & j = 1:                                              filename  ID  BMU_x  BMU_y
9   v00002_s00160(f010979-f011043)_g00246_f011028.jpg   9      0      1
39  v00015_s00029(f007898-f008573)_g00054_f008121.jpg  39      0      1
56  v00021_s00016(f008964-f009479)_g00087_f009325.jpg  56      0      1
57  v00022_s00010(f001692-f002051)_g00044_f002007.jpg  57      0      1
59  v00022_s00046(f006646-f007048)_g00144_f006953.jpg  59      0      1
df for i = 0 & j = 2:                                              filename  ID  BMU_x  BMU_